# Pipeline Unificado de Entrenamiento - Optimizado
Este notebook permite seleccionar y entrenar diferentes estrategias de modelos (FeedForward, TabNet, Mixture Density Networks, BayesianNN, Monte Carlo Dropout) sobre los mismos datos, con soporte para GPU y modularidad.

- Modularización clara: carga de datos, selección de modelo, entrenamiento, validación y evaluación.
- Selección de estrategia mediante parámetro.
- Comentarios y documentación en cada bloque.
- Importación de módulos desde directorios superiores.

**Nota:** Este pipeline reemplaza la lógica redundante de los notebooks de entrenamiento individuales, pero no elimina los originales.

In [1]:
# Configuración de entorno y paths
import sys, os
sys.path.append(os.path.abspath('..'))  # Permite importar módulos desde el directorio superior
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from classes.DataHandler import DataHandler
# Importar modelos OPTIMIZADOS
from classes.FeedForwardNetwork_opt import FeedForwardNetwork_opt
from classes.TabNet_opt import TabNetNetwork_opt
from classes.MixtureDensityNetworks_opt import MixtureDensityNetworks_opt
from classes.BayesianNN_opt import BayesianNN_opt
from classes.MonteCarloDropoutNetwork_opt import MonteCarloDropoutNetwork_opt
# Para TabNet externo
from pytorch_tabnet.tab_model import TabNetRegressor
import datetime
from classes.BaseTrainer_opt import BaseTrainer_opt


# ------------------------------
# Parámetros globales y de entrenamiento (modificables al inicio)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TARGETS = ['Mo_cumulative']
# FeedForward params:
hidden_dim = 528  # Número de neuronas en la(s) capa(s) oculta(s)
num_epochs = 1000  # Número de épocas de entrenamiento
lr = 0.001  # Tasa de aprendizaje

# -----------------------------------
# Carga y preprocesamiento de datos
# -----------------------------------
dh = DataHandler()
dh.DEVICE = DEVICE
dh.set_targets(TARGETS)
table_unified, vars_desc = dh.load_data()
inputs, targets = dh.preprocess_data(vars_desc, table_unified)
X, y, scales = dh.normalize_data(inputs, targets)
# ------
# Split con arrays, conversion a tensores SOLO después del split
X_train_np, X_val_np, y_train_np, y_val_np = train_test_split(
    X, y, test_size=0.2, random_state=42)
# Conversión a tensores directamente al DEVICE requerido
X_train = torch.tensor(X_train_np, dtype=torch.float32).to(DEVICE)
X_val   = torch.tensor(X_val_np,   dtype=torch.float32).to(DEVICE)
y_train = torch.tensor(y_train_np, dtype=torch.float32).to(DEVICE)
y_val   = torch.tensor(y_val_np,   dtype=torch.float32).to(DEVICE)

INFO:classes.DataHandler:Configuración de logging inicializada correctamente.
INFO:root:Conectado a data/Tronaduras_vs_Sismicidad.db
INFO:classes.DataHandler:Columnas objetivo actualizadas: ['Mo_cumulative']
INFO:classes.DataHandler:Inicio de la carga de datos.
INFO:classes.DataHandler:Tabla Tabla_Unificada cargada correctamente.
INFO:classes.DataHandler:Tabla Variables_Description cargada correctamente.
INFO:classes.DataHandler:Conexión cerrada.
INFO:classes.DataHandler:Datos cargados exitosamente.
INFO:classes.DataHandler:Inicio del preprocesamiento de datos.
INFO:classes.DataHandler:Variables de entrada procesadas: ['Cobertura Total', 'Cobertura Primario', 'Tronadura_Largo de Perforación (m)', 'Tronadura_N° Tiros', 'Tronadura_N° Tiros Real', 'Tronadura_Kg. de explosivos tronadura', 'Tronadura_Tipo Explosivo', 'Destressing_Se realizó', 'Destressing_N° Tiros', 'Destressing_Kg. de explosivos', 'Destressing_Tipo Explosivo', 'Geotecnicas_UCS (MPa)', 'Geotecnicas_Modulo de Young (GPa)', '

✅ Datos leídos desde 'Processed_Data.Tabla_Unificada' y convertidos a DataFrame.
✅ Datos leídos desde 'Raw_Data.Variables_Description' y convertidos a DataFrame.
🔌 Conexión cerrada manualmente.


In [2]:
# ------------------------------
model_types = ['FeedForward', 'TabNet', 'MDN', 'BayesianNN', 'MonteCarloDropout']
resultados_modelos = {}

for MODEL_TYPE in model_types:
    print(f"\nEntrenando y evaluando modelo: {MODEL_TYPE}")
    resultados = BaseTrainer_opt.run_pipeline(
        model_type=MODEL_TYPE,
        X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val,
        scales=scales, dh=dh,
        hidden_dim=hidden_dim,
        n_components=5,  # Solo relevante para MDN
        device=DEVICE,
        lr=lr,
        num_epochs=num_epochs,
        save_model=True,
        plot_results=True,
        dropout_prob=0.2  # Solo relevante para MonteCarloDropout
    )
    
    # Desempaquetar resultados según el modelo
    if MODEL_TYPE in ['FeedForward', 'TabNet']:
        y_train_mo, y_pred_train_mo, y_val_mo, y_pred_val_mo = resultados
        std_train = None
        std_val = None
    elif MODEL_TYPE == 'MDN':
        y_train_mo, y_pred_train_mo, y_val_mo, y_pred_val_mo, std_train, std_val = resultados
    elif MODEL_TYPE in ['BayesianNN', 'MonteCarloDropout']:
        y_train_mo, mean_train_mo, y_val_mo, mean_val_mo, std_train, std_val = resultados
        # mean_train_mo y mean_val_mo son las predicciones promedio
        y_pred_train_mo = mean_train_mo
        y_pred_val_mo = mean_val_mo

    resultados_modelos[MODEL_TYPE] = {
        'y_train': y_train_mo,
        'y_pred_train': y_pred_train_mo,
        'y_val': y_val_mo,
        'y_pred_val': y_pred_val_mo,
        'std_train': std_train,
        'std_val': std_val
    }





Entrenando y evaluando modelo: FeedForward
[2025-06-04 09:59:25] 🚀 [FeedForward] Entrenando red...
[2025-06-04 09:59:26] ✨ [FeedForward] Mejor modelo en época 0 📉 train loss: 0.2338 📊 val loss: 0.0214
[2025-06-04 09:59:26] 🔄 [FeedForward] [Época 0] 🧠 Pérdida entrenamiento: 0.2338 | Validación: 0.0214
[2025-06-04 09:59:26] ✨ [FeedForward] Mejor modelo en época 4 📉 train loss: 0.0345 📊 val loss: 0.0187
[2025-06-04 09:59:26] ✨ [FeedForward] Mejor modelo en época 11 📉 train loss: 0.0222 📊 val loss: 0.0156
[2025-06-04 09:59:26] ✨ [FeedForward] Mejor modelo en época 12 📉 train loss: 0.0176 📊 val loss: 0.0151
[2025-06-04 09:59:26] ✨ [FeedForward] Mejor modelo en época 20 📉 train loss: 0.0150 📊 val loss: 0.0145
[2025-06-04 09:59:26] ✨ [FeedForward] Mejor modelo en época 27 📉 train loss: 0.0144 📊 val loss: 0.0143
[2025-06-04 09:59:26] ✨ [FeedForward] Mejor modelo en época 40 📉 train loss: 0.0126 📊 val loss: 0.0143
[2025-06-04 09:59:26] ✨ [FeedForward] Mejor modelo en época 44 📉 train loss: 0.0

KeyboardInterrupt: 